# `configurable_fields` method

In [15]:
from langchain_core.runnables import ConfigurableField
from langchain_community.llms import Ollama

model = Ollama(model="mistral", temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [17]:
# first try at temperature 0
model.invoke("Pick a number. The answer should contain only one digit.")

" I cannot pick a number myself, but here's an example: The answer to the sum of 5 and the number you have in mind is 6. So, the number you thought of is 1."

In [18]:
# second try at temperature 0
model.invoke("Pick a number. The answer should contain only one digit.")

" I cannot pick a number myself, but here's an example: The answer to the sum of 5 and the number you have in mind is 6. So, the number you thought of is 1."

In [13]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("Pick a number. The answer should contain only one digit.")

' I cannot choose a number with only one digit without knowing which specific number you have in mind. However, if we follow the instruction and pick any random number, let\'s go with the number "7" since it is a single-digit number when considering the ones place of a two-digit number (for example, 17). So, the answer will be \'7\'.'

## With `HubRunnables`

In [19]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

In [20]:
prompt.invoke({"question": "foo", "context": "bar"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: foo \nContext: bar \nAnswer:")])

In [21]:
config={"hub_commit": "rlm/rag-prompt-llama"}
input={"question": "foo", "context": "bar"}

prompt.with_config(configurable=config).invoke(input)

ChatPromptValue(messages=[HumanMessage(content="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: foo \nContext: bar \nAnswer: [/INST]")])

# `configurable_alternatives` method

## With `LLMs`

In [22]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import ConfigurableField

In [23]:
llm = ChatOllama(model="mistral", temperature=0).configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key="mistral",
    llama=ChatOllama(model="llama2")
)

prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm

In [24]:
chain.invoke({"topic": "bears"})

AIMessage(content=' Why don\'t bears like to be bothered in the morning? because they are grrrrrumpy before their coffee!\n\nOr, why did the bear go to the party? Because he heard there would be a lot of "B"s there: Bar-B-Q, Beats, and Beautiful people! \n\nI hope you found those jokes amusing! Let me know if you\'d like to hear any more.')

In [25]:
chain.with_config(configurable={"llm": "llama"}).invoke({"topic": "bears"})

AIMessage(content='Why did the bear go to the restaurant?\n\nBecause he wanted to have a "bear"-ful time!')

## With `Prompts`

In [46]:
prompt = PromptTemplate.from_template("Tell me a joke about {topic}").configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="joke",
    poem=PromptTemplate.from_template("Write a short poem about {topic}")
)

In [47]:
prompt.invoke({"topic": "bears"})

StringPromptValue(text='Tell me a joke about bears')

In [48]:
prompt.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

StringPromptValue(text='Write a short poem about bears')